In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


plt.style.use('ggplot')

import nltk
import re
from nltk.corpus import stopwords
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [17]:
# Read in data
df = pd.read_csv('test.csv', encoding = 'latin1')
df.head()

,Unnamed: 0,ID,likeCount,replyCount,retweetCount,Tweet,label
0,2550,612,6,0,1,TODAY...GAME DAY..Ã°ÂÂÂBEST OF LUCK CUTIE P...,1
1,2551,1772,0,0,0,@CricketAus #SAvAUS yes finally justice for Hu...,1
2,2552,2336,0,0,0,Good luck for todays match MSD &amp; Co. !! #I...,1
3,2553,957,0,0,0,@BLACKCAPS Such a disgusting,0
4,2554,1633,0,0,0,"""Umar Amin has a lot of potential and he playe...",1


In [3]:
df.shape

(450, 7)

In [4]:
df['label'].value_counts()

1    369
0     81
Name: label, dtype: int64

##  VADER sentiment scoring


In [12]:
from nltk.sentiment import SentimentIntensityAnalyzer
from tqdm.notebook import tqdm

sia = SentimentIntensityAnalyzer()

In [18]:
# Create Prediction column based on Polarity Score
df['Prediction'] = df['Tweet'].apply(lambda x: 1 if sia.polarity_scores(x)['compound'] >= 0 else -1)
df.head()

,Unnamed: 0,ID,likeCount,replyCount,retweetCount,Tweet,label,Prediction
0,2550,612,6,0,1,TODAY...GAME DAY..Ã°ÂÂÂBEST OF LUCK CUTIE P...,1,1
1,2551,1772,0,0,0,@CricketAus #SAvAUS yes finally justice for Hu...,1,1
2,2552,2336,0,0,0,Good luck for todays match MSD &amp; Co. !! #I...,1,1
3,2553,957,0,0,0,@BLACKCAPS Such a disgusting,0,-1
4,2554,1633,0,0,0,"""Umar Amin has a lot of potential and he playe...",1,1


In [19]:
# Edit Label column 1 for Positive, -1 for Negative
df['label'] = df['label'].apply(lambda x: -1 if x == 0 else 1)
df.head()

,Unnamed: 0,ID,likeCount,replyCount,retweetCount,Tweet,label,Prediction
0,2550,612,6,0,1,TODAY...GAME DAY..Ã°ÂÂÂBEST OF LUCK CUTIE P...,1,1
1,2551,1772,0,0,0,@CricketAus #SAvAUS yes finally justice for Hu...,1,1
2,2552,2336,0,0,0,Good luck for todays match MSD &amp; Co. !! #I...,1,1
3,2553,957,0,0,0,@BLACKCAPS Such a disgusting,-1,-1
4,2554,1633,0,0,0,"""Umar Amin has a lot of potential and he playe...",1,1


In [23]:
# Check if the Label column and Prediction column match for accuracy calculation
df['Accuracy'] = df.apply(lambda x: 1 if x[6] == x[7] else 0, axis=1)
df.head()

,Unnamed: 0,ID,likeCount,replyCount,retweetCount,Tweet,label,Prediction,Accuracy
0,2550,612,6,0,1,TODAY...GAME DAY..Ã°ÂÂÂBEST OF LUCK CUTIE P...,1,1,1
1,2551,1772,0,0,0,@CricketAus #SAvAUS yes finally justice for Hu...,1,1,1
2,2552,2336,0,0,0,Good luck for todays match MSD &amp; Co. !! #I...,1,1,1
3,2553,957,0,0,0,@BLACKCAPS Such a disgusting,-1,-1,1
4,2554,1633,0,0,0,"""Umar Amin has a lot of potential and he playe...",1,1,1


In [25]:
def conf_matrix(x):
  if x[6] == 1 and x[7] == 1:
    return 'TP'
  elif x[6] == 1 and x[7] == -1:
    return 'FN'
  elif x[6] == -1 and x[7] == 1:
    return 'FP'
  elif x[6] == -1 and x[7] == -1:
    return 'TN'
  else:
    return 0
    
df['Conf_Matrix'] = df.apply(lambda x: conf_matrix(x), axis=1)
df.head()

,Unnamed: 0,ID,likeCount,replyCount,retweetCount,Tweet,label,Prediction,Accuracy,Conf_Matrix
0,2550,612,6,0,1,TODAY...GAME DAY..Ã°ÂÂÂBEST OF LUCK CUTIE P...,1,1,1,TP
1,2551,1772,0,0,0,@CricketAus #SAvAUS yes finally justice for Hu...,1,1,1,TP
2,2552,2336,0,0,0,Good luck for todays match MSD &amp; Co. !! #I...,1,1,1,TP
3,2553,957,0,0,0,@BLACKCAPS Such a disgusting,-1,-1,1,TN
4,2554,1633,0,0,0,"""Umar Amin has a lot of potential and he playe...",1,1,1,TP


In [27]:
conf_vals = df.Conf_Matrix.value_counts().to_dict()
print(conf_vals)

accuracy = (conf_vals['TP'] + conf_vals['TN']) / (conf_vals['TP'] + conf_vals['TN'] + conf_vals['FP'] + conf_vals['FN'])
precision = conf_vals['TP'] / (conf_vals['TP'] + conf_vals['FP'])
recall = conf_vals['TP'] / (conf_vals['TP'] + conf_vals['FN'])
f1_score = 2*precision*recall / (precision + recall)
print('Accuracy: ', round(100 * accuracy, 2),'%',
      '\nPrecision: ', round(100 * precision, 2),'%',
      '\nRecall: ', round(100 * recall, 2),'%',
      '\nF1 Score: ', round(100 * f1_score, 2),'%')

{'TP': 353, 'TN': 45, 'FP': 36, 'FN': 16}
Accuracy:  88.44 % 
Precision:  90.75 % 
Recall:  95.66 % 
F1 Score:  93.14 %
